In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/pytorchfordeeplearning/
!python download.py

/content/drive/MyDrive/pytorchfordeeplearning
Creating directory ./train/cat
Creating directory ./train/fish
Creating directory ./test/cat
Creating directory ./test/fish
Creating directory ./val/cat
Creating directory ./val/fish
Downloaded http://farm2.static.flickr.com/1245/1259825348_6a2aa94e8d.jpg to train/cat/1259825348_6a2aa94e8d.jpg
Downloaded http://farm2.static.flickr.com/1080/1029412358_7ee17550fc.jpg to train/cat/1029412358_7ee17550fc.jpg
Downloaded http://farm1.static.flickr.com/196/443645811_8c4bb1af50.jpg to train/cat/443645811_8c4bb1af50.jpg
Downloaded http://farm2.static.flickr.com/1201/1285591549_593ca7cf6a.jpg to train/cat/1285591549_593ca7cf6a.jpg
Downloaded http://farm3.static.flickr.com/2002/1760479467_068432dd3f.jpg to train/cat/1760479467_068432dd3f.jpg
Downloaded http://farm2.static.flickr.com/1411/649938210_e9dcbde5ea.jpg to train/cat/649938210_e9dcbde5ea.jpg
Downloaded http://farm2.static.flickr.com/1423/918775832_9ecfd414b8.jpg to train/cat/918775832_9ecfd414b

In [93]:
import torchvision
from torchvision import transforms

train_data_path = "/content/drive/MyDrive/pytorchfordeeplearning/train"
val_data_path = "/content/drive/MyDrive/pytorchfordeeplearning/val"
test_data_path = "/content/drive/MyDrive/pytorchfordeeplearning/test"

img_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(
    root = train_data_path, transform=img_transforms
)

val_data = torchvision.datasets.ImageFolder(
    root = val_data_path, transform=img_transforms
)

test_data = torchvision.datasets.ImageFolder(
    root = test_data_path, transform=img_transforms
)

In [245]:
## Building our data loaders
import torch
from torch.utils import data

batch_size = 229

# Create data loaders
train_data_loader = data.DataLoader(train_data, batch_size = batch_size)

val_data_loader = data.DataLoader(val_data, batch_size = batch_size)

test_data_loader = data.DataLoader(test_data, batch_size = batch_size)

In [260]:
# Creating the network.
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):

  def __init__(self):
    super(SimpleNet, self).__init__()
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50, 20)
    self.fc4 = nn.Linear(20, 2)

  def forward(self, x):
    x = x.view(-1, 12288)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x


simplenet = SimpleNet()

In [261]:
import torch.optim as optim
optimizer = optim.Adam(simplenet.parameters(), lr=0.0001)

In [262]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20):
  for epoch in range(epochs):
    training_loss = 0.0
    valid_loss = 0.0
    model.train()

    for batch in train_loader:
      optimizer.zero_grad()
      input, target = batch
      output = model(input)
      loss = loss_fn(output, target)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item()
    training_loss /= len(train_loader)

    model.eval()
    num_correct = 0
    num_examples = 0
    for batch in val_loader:
      inputs, targets = batch
      output = model(inputs)
      loss = loss_fn(output, targets)
      valid_loss += loss.data.item()
      correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0]
      valid_loss /= len(val_loader.dataset)

    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))




In [263]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=50)

Epoch: 0, Training Loss: 0.68, Validation Loss: 0.01, accuracy = 0.70
Epoch: 1, Training Loss: 0.62, Validation Loss: 0.01, accuracy = 0.67
Epoch: 2, Training Loss: 0.60, Validation Loss: 0.01, accuracy = 0.66
Epoch: 3, Training Loss: 0.58, Validation Loss: 0.01, accuracy = 0.65
Epoch: 4, Training Loss: 0.56, Validation Loss: 0.01, accuracy = 0.66
Epoch: 5, Training Loss: 0.55, Validation Loss: 0.01, accuracy = 0.68
Epoch: 6, Training Loss: 0.53, Validation Loss: 0.01, accuracy = 0.68
Epoch: 7, Training Loss: 0.52, Validation Loss: 0.01, accuracy = 0.69
Epoch: 8, Training Loss: 0.51, Validation Loss: 0.01, accuracy = 0.69
Epoch: 9, Training Loss: 0.49, Validation Loss: 0.01, accuracy = 0.69
Epoch: 10, Training Loss: 0.48, Validation Loss: 0.01, accuracy = 0.70
Epoch: 11, Training Loss: 0.47, Validation Loss: 0.01, accuracy = 0.70
Epoch: 12, Training Loss: 0.46, Validation Loss: 0.01, accuracy = 0.69
Epoch: 13, Training Loss: 0.45, Validation Loss: 0.01, accuracy = 0.69
Epoch: 14, Train

In [264]:
import torch
import torch.nn.functional as F

# Set the model to evaluation mode
simplenet.eval()

labels = ['cat', 'fish']

correct_predictions = 0
total_images = 0

with torch.no_grad():
    for batch in test_data_loader:
        inputs, targets = batch

        outputs = simplenet(inputs)

        predictions = torch.argmax(outputs, dim=1)

        correct_predictions += (predictions == targets).sum().item()
        total_images += targets.size(0)


accuracy = correct_predictions / total_images * 100
print(f"Accuracy on the test set: {accuracy:.2f}%")


Accuracy on the test set: 76.62%


In [265]:
torch.save(simplenet.state_dict(), "/content/drive/MyDrive/pytorchfordeeplearning/simplenet")
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/content/drive/MyDrive/pytorchfordeeplearning/simplenet")
simplenet.load_state_dict(simplenet_state_dict)

<ipython-input-265-d0a084fa3aa7>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  simplenet_state_dict = torch.load("/content/drive/MyDrive/pytorchfordeeplearning/simplenet"

<All keys matched successfully>